In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)  # torch tensor with grad

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0  # python number accumulated over whole dataset, so need to be initilize to zero

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()  # python number
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")  # right align a string in Python f-strings using {variable:>N} where N is the total length

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300740  [    0/60000]
loss: 2.289685  [ 6400/60000]
loss: 2.275322  [12800/60000]
loss: 2.271771  [19200/60000]
loss: 2.267366  [25600/60000]
loss: 2.229327  [32000/60000]
loss: 2.253539  [38400/60000]
loss: 2.224809  [44800/60000]
loss: 2.221435  [51200/60000]
loss: 2.201210  [57600/60000]
Test Error: 
 Accuracy: 37.3%, Avg loss: 0.034662 

Epoch 2
-------------------------------
loss: 2.214190  [    0/60000]
loss: 2.207260  [ 6400/60000]
loss: 2.162433  [12800/60000]
loss: 2.182727  [19200/60000]
loss: 2.171232  [25600/60000]
loss: 2.089320  [32000/60000]
loss: 2.154831  [38400/60000]
loss: 2.091708  [44800/60000]
loss: 2.105784  [51200/60000]
loss: 2.059129  [57600/60000]
Test Error: 
 Accuracy: 42.9%, Avg loss: 0.032608 

Epoch 3
-------------------------------
loss: 2.091804  [    0/60000]
loss: 2.074235  [ 6400/60000]
loss: 1.987825  [12800/60000]
loss: 2.040796  [19200/60000]
loss: 2.015483  [25600/60000]
loss: 1.885004  [32000/600